In [ ]:
#运行本notebook需要以下库，如本cell报错，请先安装库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv
import math
from sklearn.model_selection import KFold
from scipy.io import loadmat

In [ ]:
#参数表，能够通过调整这些参数改变模型运算的性能
#隐藏神经元的数量，可改变来改变隐藏层的规模
center_point_number=720 
#用于训练集的mat文件
mat_path = '测试数据(小组)\\train.mat'

#用于测试集的mat文件
mat_path_test = '其他资料\测试数据(所有人)\\3\\train.mat'

In [ ]:
#数据预处理函数
def matimport(path): #返回训练值和标记值
    m = loadmat(path)
    return m['feature'],m['label']

#输出标签数据处理函数
def Output_Matrix(input_y):#对label值进行一些调整，把0变为[1,0,0],1变[0,1,0],2变[0,0,1]
    size=input_y.shape[0]           
    op_y=np.zeros((size,3))
    for i in range(size):
        if input_y[i]==0:
            op_y[i][0]=1
        elif input_y[i]==1:
            op_y[i][1]=1
        else:
            op_y[i][2]=1
    return op_y

In [ ]:
#有关计算函数
def MED(point1,point2):         #计算两点之间的欧式距离
    return (np.sum((point1-point2)**2))**0.5
    
def Gaussian(point1,point2,variance):  #径向基函数、高斯函数
    return math.exp(-(MED(point1,point2))**2/(2*variance**2))

def get_variance(center_point_number,center_point):   #计算方差
    max_MED=0
    for i in range(center_point_number):
        for j in range(center_point_number):
            med_=MED(center_point[i],center_point[j])
            if med_>max_MED:
                max_MED=med_
    return max_MED/((2*center_point_number)**0.5)

def get_wight(ip_x,center_point,center_point_number,op_y,variance):          #计算权值矩阵
    weight=np.zeros((center_point_number,3))      #权重矩阵
    j_matrix=np.zeros((center_point_number,ip_x.shape[1])) #激励矩阵
    for i in range(center_point_number):
        for j in range(ip_x.shape[1]):
            j_matrix[i][j]=Gaussian(center_point[i],ip_x[j], variance)
    j_inverse=np.linalg.pinv(j_matrix)    #激励矩阵的伪逆
    weight=np.dot(j_inverse.T,op_y)
    return weight 